# A simple chatbot agent

## Prepare the environment

In [1]:
import sys
sys.path.append('../../..')

from autogen import ConversableAgent, register_function

# from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent
from inswitch.util import second_last_msg
from inswitch.agent.rdf import RdfAgent

from rdflib import Graph, URIRef

## The functions to query the complete kg
TODO: generalize it to work on given ttl files.

## Here are the API invocation agents

In [2]:
rdf_tool_driver_system_message = '''
From a user intent (provided in the context), you may figure out what you can get from the knowledge graph.
You may do it step by step, starting from listing all the service descriptions, 
and see which exact service the intent is about. 
After that you can find the containers that need to deployed to the target active_machine (as mentioned in the intend),
to implement the service that is needed.
'''
kg = Graph()
kg.parse("../data/anonimized_data_fill.ttl")

rdf_agent = RdfAgent(
    "rdf_agent",
    graph=kg, 
    system_message=rdf_tool_driver_system_message,
    max_internal_turns=6
)

def get_service_descriptions()->str:
    return """
        PREFIX intend: <https://intendproject.eu/schema/>
        SELECT ?subject ?predict ?object 
        WHERE{?subject ?predict ?object FILTER(?predict = intend:description)}
        """

def get_containers_to_deploy(service:str, machine:str)->str:
    return f""" 
    PREFIX intend: <https://intendproject.eu/schema/>
    SELECT ?container ?predict ?ip
    WHERE{{
        {service} intend:hasComponent ?component .
        ?container intend:isContainerOf ?component .
        ?machinetype intend:hasService {service} .
        {machine} intend:is_machine_type ?machinetype .
        {machine} intend:has_IP ?ip 
        BIND(intend:deploy as ?predict).
    }}
    """


rdf_agent.register_fixed_query(
    get_service_descriptions, 
    "This function can extract all the service descriptions"
)

# rdf_agent.register_fixed_query(
#     get_containers_to_deploy,
#     "this function finds out the containers to deploy on a machine, to implement the target service"
# )


## Here we go!
A simple sequence: user_intent -> api_doc_provider -> nerv_tool_executor

Current intent is hard coded.

In [ ]:
user_intent = '''I am Customer1 and own Active_machine6. 
I want to add a new service to my machine to record all the alarms
'''

intent_provider = get_fixed_reply_agent(
    'intent_provier',
    reply = user_intent
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)

decision_chat_results = moderator.initiate_chats(
    [
        {
            "recipient": intent_provider,
            "message": "what do you want?",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": rdf_agent,
            "message": f"",
            "max_turns": 1,
            # "summary_method": "last_msg"
        }
    ]
)




********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to intent_provier):

what do you want?

--------------------------------------------------------------------------------
intent_provier (to moderator):

I am Customer1 and own Active_machine6. 
I want to add a new service to my machine to record all the alarms


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to rdf_agent):


Context: 
I am Customer1 and own Active_machine6. 
I want to add a new service to my machine to record all the alarms


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

*********

## Next steps:

1. Replace current api_doc_provider (a fixed_reply_agent) by a RAG assistant, that extracts from a complete API document the parts that are relevant to DNA and deployment.
2. Add in the beginning of the sequence another agent (or agents) to extract the list of workloads. Eventually, the input intent should be like "I want to be able to monitor the thermal stability of machine M0001"